In [64]:
import gymnasium as gym
import random
import time

In [2]:
desc=["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False)
observation, info = env.reset()
#step result:
#(square, reward, end, idk, idk)

In [ ]:
#panic switch
env.close()

In [3]:
def create_Q_table():
    q_table = {}
    # Populate Q-table with zeros for each state-action pair
    for i in range(20):
        q_table[i] = {0: 0, 1: 0, 2: 0, 3: 0}
    return q_table
        
#L = 0, D = 1, R = 2, U = 3

In [74]:
#init vars
alpha = 0.5 #α
gamma = 0.5 #γ
epsilon = 0.3 #ε - chance of random
episodeMax = 100000
num_columns = 4
num_rows = 5

In [5]:
grid = [[1, 2, 3, 4],
        [5, 6, 7, 8],
        [9, 10, 11, 12],
        [13, 14, 15, 16],
        [17, 18, 19, 20]]

In [ ]:
#q_table[square][direction] = q_table[square][direction] + alpha * (reward + gamma * my_max(q_table[newSquare]) - q_table[square][direction])

In [ ]:
#(q_table[square][0] == q_table[square][1]) and (q_table[square][2] == q_table[square][3])

In [6]:
def find_row_col(grid, value):
    for row_index, row in enumerate(grid):
        if value in row:
            col_index = row.index(value)
            return row_index, col_index

In [7]:
def my_max(table):
    #{0: 0, 1: 0.0, 2: 0.0, 3: 0}
    maxNum = table[0]
    maxID = 0
    for i in range(1,4):
        num = table[i]
        if(num > maxNum):
            maxNum = num
            maxID = i
    return maxID

In [11]:
def totalDiff(q_table):
    total = 0
    for row in q_table.values():
        for value in row.values():
            total += value
    return total

In [75]:
q_table = create_Q_table()
start_time = time.time()
for i in range(episodeMax):#looping through episodes
    terminal = False
    observation, info = env.reset()
    square = 0
    total = totalDiff(q_table)
    while(not terminal):
        allSame = (q_table[square][0] == q_table[square][1]) and (q_table[square][2] == q_table[square][3])
        if(random.randint(0, 10) > epsilon * 10 and not allSame): #check if all values are same 
            direction = my_max(q_table[square])
        else:
            direction = random.randint(0, 3)
        row, col = find_row_col(grid, square+1)  
        if(direction == 0 and col == 0):
            continue
        elif(direction == 1 and row == 4):
            continue
        elif(direction == 2 and col == 3):
            continue
        elif(direction == 3 and row == 0):
            continue
        arr = env.step(direction)
        newSquare = arr[0]
        reward = arr[1]
        newQValue = q_table[square][direction] + alpha * (reward + gamma * q_table[newSquare][my_max(q_table[newSquare])] - q_table[square][direction])
        q_table[square][direction] = newQValue
        terminal = arr[2]
        square = newSquare
    newTotal = totalDiff(q_table)
    if(abs(newTotal - total) > 0 and abs(newTotal - total)< 0.01):
        print(i)
        break
end_time = time.time()    
q_table

159


{0: {0: 0, 1: 0.028774917125701904, 2: 0.0, 3: 0},
 1: {0: 0.0, 1: 0.0, 2: 0.0, 3: 0},
 2: {0: 0.0, 1: 0.0, 2: 0.0, 3: 0},
 3: {0: 0.0, 1: 0.0, 2: 0, 3: 0},
 4: {0: 0, 1: 0.059603214263916016, 2: 0.0, 3: 0.012963414192199707},
 5: {0: 0, 1: 0, 2: 0, 3: 0},
 6: {0: 0, 1: 0, 2: 0, 3: 0},
 7: {0: 0, 1: 0, 2: 0, 3: 0},
 8: {0: 0, 1: 0.0, 2: 0.12326955795288086, 3: 0.0},
 9: {0: 0.03904247283935547, 1: 0.24914264678955078, 2: 0.0, 3: 0.0},
 10: {0: 0, 1: 0, 2: 0.0, 3: 0.0},
 11: {0: 0.0, 1: 0.0, 2: 0, 3: 0},
 12: {0: 0, 1: 0, 2: 0, 3: 0},
 13: {0: 0, 1: 0.4998302459716797, 2: 0.0, 3: 0.05126953125},
 14: {0: 0, 1: 0, 2: 0, 3: 0},
 15: {0: 0, 1: 0, 2: 0, 3: 0.0},
 16: {0: 0, 1: 0, 2: 0, 3: 0},
 17: {0: 0, 1: 0, 2: 0.99993896484375, 3: 0.2284832000732422},
 18: {0: 0, 1: 0, 2: 0, 3: 0},
 19: {0: 0, 1: 0, 2: 0, 3: 0}}

In [76]:
elapsed_time = end_time - start_time
print("Time elapsed:", elapsed_time, "seconds")

Time elapsed: 0.02099919319152832 seconds


In [56]:
env.close()

In [53]:
envTwo = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human")
observation, info = envTwo.reset()

In [61]:
observation, info = envTwo.reset()
terminal = False
square = 0
while(not terminal):
    allSame = (q_table[square][0] == q_table[square][1]) and (q_table[square][2] == q_table[square][3])
    if(not allSame): #check if all values are same 
        direction = my_max(q_table[square])
    else:
        direction = random.randint(0, 3)
    row, col = find_row_col(grid, square+1)  
    if(direction == 0 and col == 0):
        continue
    elif(direction == 1 and row == 4):
        continue
    elif(direction == 2 and col == 3):
        continue
    elif(direction == 3 and row == 0):
        print(direction == 3 and row == 0) 
        continue
    arr = envTwo.step(direction)
    newSquare = arr[0]
    reward = arr[1]
    terminal = arr[2]
    square = newSquare

In [62]:
envTwo.close()